### This code still need fix

In [1]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pandas as pd
import pickle
import yaml

from lib.notebooks import plot_groups_metrics
from imitative_agent import ImitativeAgent

current path: /mnt/c/Users/vpaul/OneDrive - CentraleSupelec/Inner_Speech/agent/imitative_agent_inner_speech_silent_finetune


In [2]:
agents_path = glob("../out/imitative_agent_inner_speech_silent_finetune_nojerk7/*/")
agents_path.sort()
print(f"Found {len(agents_path)} agents")

Found 30 agents


In [4]:
# Dictionary to store metrics for different groups of agents
groups_metrics = {}

# Dictionary to store various parameters and performance metrics for each agent
agents_loss = {
    "path": [],                        # Path to agent directory
    "datasets": [],                    # Training datasets used
    "learning_rate": [],              # Learning rate
    "inverse_layers": [],             # Layer configuration of inverse model
    "inverse_dropout_p": [],          # Dropout probability for inverse model
    "direct_layers": [],              # Layer configuration of direct model
    "direct_dropout_p": [],           # Dropout probability for direct model
    "jerk_weight": [],               # Weight of jerk loss term
    "direct_estimation_error": [],    # Test error of direct model
    "inverse_estimation_error": [],   # Test error of inverse model
    "jerk": [],                      # Jerk metric on test set
    "repetition_error": [],          # Repetition error on test set
}

# Iterate through all agent directories to collect metrics and parameters
for agent_path in agents_path:
    # Load agent configuration without neural networks for efficiency
    agent = ImitativeAgent.reload(agent_path, load_nn=False)
    config = agent.config
    
    # Load stored metrics from training
    with open("%s/metrics.pickle" % agent_path, "rb") as f:
        metrics = pickle.load(f)
    
    # Store basic agent information
    agents_loss["path"].append(agent_path[-5:-1])
    agents_loss["datasets"].append(",".join(config['dataset']['names']))

    # Store model parameters
    agents_loss["learning_rate"].append(config['training']['learning_rate'])
    agents_loss["inverse_layers"].append(f"{config['model']['inverse_model']['num_layers']}x{config['model']['inverse_model']['hidden_size']}")
    agents_loss["inverse_dropout_p"].append(config['model']['inverse_model']['dropout_p'])
    agents_loss["direct_layers"].append(f"{len(config['model']['direct_model']['hidden_layers'])}x{config['model']['direct_model']['hidden_layers'][0]}")
    agents_loss["direct_dropout_p"].append(config['model']['direct_model']['dropout_p'])
    agents_loss["jerk_weight"].append(config['training']['jerk_loss_weight'])

    # Find index of best validation performance for reporting test metrics
    final_loss_index = np.argmin(metrics["validation"]["inverse_model_repetition_error"])
    
    # Store test performance metrics
    agents_loss["inverse_estimation_error"].append(metrics["test"]["inverse_model_estimation_error"][final_loss_index])
    agents_loss["jerk"].append(metrics["test"]["inverse_model_jerk"][final_loss_index])
    agents_loss["repetition_error"].append(metrics["test"]["inverse_model_repetition_error"][final_loss_index])
    
    # Create group name based on key configuration parameters
    group_name = "\n".join((
        f"datasets={','.join(config['dataset']['names'])}",
        f"synth={config['synthesizer']['name']}",
        f"jerk_w={config['training']['jerk_loss_weight']}",
    ))
    
    # Store metrics for this group
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][agent_path] = metrics

# Convert collected data to pandas DataFrame for easier analysis
agents_loss = pd.DataFrame(agents_loss)

{'dataset': {'batch_size': 8, 'datasplits_size': [64, 16, 20], 'names': ['pb2007'], 'num_workers': 6, 'shuffle_between_epochs': True, 'sound_type': 'cepstrum'}, 'model': {'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 0.25, 'hidden_size': 32, 'num_layers': 2}}, 'synthesizer': {'name': 'ea587b76c95fecef01cfd16c7f5f289d-0/'}, 'training': {'jerk_loss_ceil': 0, 'jerk_loss_weight': 0, 'learning_rate': 0.001, 'max_epochs': 800, 'patience': 100, 'vel_loss_ceil': 0, 'vel_loss_weight': 0}}
{'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 0.25, 'hidden_size': 32, 'num_layers': 2}}
{'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 

ValueError: All arrays must be of the same length

In [4]:
# Extract unique dataset names and jerk weight values from the agents_loss DataFrame
datasets = pd.unique(agents_loss["datasets"])
jerk_weights = pd.unique(agents_loss["jerk_weight"])

def show_top_agents(measure="repetition_error", datasets=datasets[0], jerk_weight=jerk_weights[0], 
                   ascending=True):
    """
    Display top performing agents based on specified criteria and performance measure.
    
    Args:
        measure (str): Performance metric to sort by (default: "repetition_error")
        datasets (str): Dataset name to filter results
        jerk_weight (float): Jerk weight value to filter results  
        ascending (bool): Sort order - True for ascending (better), False for descending
    
    Returns:
        Displays DataFrame of at most top 30 agents matching criteria, sorted by performance measure
    """
    # Filter agents by selected dataset
    subset = agents_loss[agents_loss["datasets"] == datasets]
    # Further filter by jerk weight parameter
    subset = subset[subset["jerk_weight"] == jerk_weight]
    
    # Display at most top 30 agents sorted by the specified measure
    display(subset.sort_values(measure, ascending=ascending).head(30))

# Create interactive widget to explore agent performances
ipw.interactive(show_top_agents, 
                measure=agents_loss.columns,  # Available performance metrics
                datasets=datasets,    # Dataset options
                jerk_weight=jerk_weights,  # Jerk weight options
                ascending=True)       # Toggle sort order

In [5]:
# Define key performance metrics to visualize
metrics_name = [
    # "direct_model_estimation_error",         # Error in direct model predictions
    "inverse_model_estimation_error",        # Error in inverse model parameter estimation
    "inverse_model_repetition_error",        # Consistency error across repeated trials
    "inverse_model_jerk",                    # Smoothness of generated trajectories
]

def show_metrics(split_name="test"):
    """
    Visualize training metrics across different data splits.
    
    Args:
        split_name (str): Data split to analyze - "train", "validation", or "test"
                         Defaults to "test" split.
    
    Displays:
        Interactive plot showing the progression of selected metrics over training
        for the specified data split. Metrics are color-coded and labeled for clarity.
    """
    plot_groups_metrics(groups_metrics, metrics_name, split_name)

# Create interactive widget to toggle between different data splits
display(ipw.interactive(show_metrics, split_name=["train", "validation", "test"]))